In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib qt5

In [2]:
import narsil
from narsil.segmentation.run import loadNet, segmentPosDirectory
from torchvision import transforms
import matplotlib.pyplot as plt
from skimage.io import imread
import torch
from narsil.liverun.utils import resizeOneImage, tensorizeOneImage
import numpy as np
from scipy.signal import find_peaks
from pathlib import Path
import glob

In [3]:
path = Path("C:\\Users\\Praneeth\\Documents\\Elflab\\Code\\testdata\\analysisData\\")

In [4]:
filenames = list(path.glob("*.tiff"))

In [5]:
filenames[0]

WindowsPath('C:/Users/Praneeth/Documents/Elflab/Code/testdata/analysisData/0_0.tiff')

In [6]:
phase_filename = Path("C:\\Users\\Praneeth\\Documents\\Elflab\\Code\\testdata\\hetero40x\\Pos100\\phaseFast\\img_000000000.tiff")

In [7]:
filenames[0]

WindowsPath('C:/Users/Praneeth/Documents/Elflab/Code/testdata/analysisData/0_0.tiff')

In [8]:
mask_img = imread(filenames[0])
phase_img = imread(phase_filename)

In [9]:
plt.imshow(phase_img, cmap='gray')

In [10]:
plt.imshow(mask_img)

### Generating histograms to cut channels and blocks

In [65]:
def getAllBarcodes(mask_img, phase_image, minPeaksDistance=25, barcodeWidth=48,
                   minChannelLength=100, rowThreshold=10000,
                   plot=False):
    hist = np.sum(mask_img, axis = 0) > minChannelLength
    peaks, _ = find_peaks(hist, distance = minPeaksDistance)
    indices_with_larger_gaps = np.where(np.ediff1d(peaks) > barcodeWidth)[0]
    
    locations_before_barcode = peaks[indices_with_larger_gaps]
    locations_after_barcode = peaks[indices_with_larger_gaps + 1]
    
    # take the mean to obtain approximate center of the barcode region
    
    locations_barcode = np.rint(np.mean((locations_before_barcode,
                                        locations_after_barcode), axis=0)).astype('int')
    
    # now grab all the barcode images
    # we get the column numbers from the locations_barcode
    # but we need to find the row numbers, somehow. 
    # We use the same histogram and sum and threshold 
    sum_rows = np.sum(mask_img, axis = 1).astype('int')
    row_locations = np.argwhere(np.diff(np.sign(sum_rows - rowThreshold))).flatten()
    if len(row_locations) !=2:
        row_x1 = 400
        row_x2 = 1200
    else:
        row_x1 = row_locations[0]
        row_x2 = row_locations[1]
    # on 40x the channels size will be approximately.
    
    barcode_images = []
    for location in locations_barcode:
        barcode_img = phase_img[row_x1:row_x2, location-barcodeWidth//2 : location+barcodeWidth//2]
        barcode_images.append(barcode_img)
        
    if plot:
        plt.figure(figsize=(16,5))
        plt.plot(hist)
        plt.plot(peaks, hist[peaks],'r*')
        plt.plot(locations_before_barcode, hist[locations_before_barcode], 'k*')
        plt.plot(locations_after_barcode, hist[locations_after_barcode], 'g*')
        plt.show()
    return locations_barcode, barcode_images

def getAllChannels():
    pass

In [66]:
locations_barcode, barcode_images = getAllBarcodes(mask_img, phase_img, plot=True)

In [69]:
plt.imshow(barcode_images[2])